## Migration Testing - Leonardo

#### Read in Currents feed

In [1]:
from dotenv import load_dotenv
from currentsapi import CurrentsAPI
import os
import requests
import time
import openai
import random

In [2]:
load_dotenv()
currents_api_key = os.environ['currents_api_key']
leonardo_api_key = os.environ['leonardo_key']
openai.api_key = os.environ['openai_key']

In [3]:
api = CurrentsAPI(api_key=currents_api_key)

In [4]:
news_feed = api.search(country="", language="en", category="world")

#### Process into GPT prompt

In [5]:
titles = [x['title'] for x in news_feed['news']]

In [6]:
titles

['Cuba Thwarts Terror Plot by South Florida Man Who Arrived by Jetski, Report Says - News18',
 "'Reiterated India's Long-Standing Position': Jaishankar After Speaking to Palestinian PM on Gaza Crisis - News18",
 'Hong Kong holds first council elections under new rules that shut out pro-democracy candidates',
 'Philippines, China trade accusations over South China Sea collision',
 "Guyana and Venezuela's Territorial Dispute: What to Know",
 'US Business Leaders Rally Around Indian American GOP Candidate Nikki Haley Over Trump - News18',
 'Palestinians report Israeli battles in Khan Younis after US blocks Gaza ceasefire call',
 'France detains 80 in largest child sex crime swoop',
 "Philippine supply boat 'rammed' by China Coast Guard vessel, says official",
 'Democrat John Whitmire Elected Houston Mayor',
 "China's Xi looks to strengthen Vietnam ties after Biden visit",
 "Hong Kong Holds First 'Patriots Only' Local Elections - News18",
 'Cuba thwarts terrorist plot by Florida man who ar

In [7]:
selected_snippet = random.choice(titles)
#selected_snippet= '''China Protests Calls to Include Taiwan in Climate Talks'''

In [8]:
selected_snippet

'Palestinians report Israeli battles in Khan Younis after US blocks Gaza ceasefire call'

In [9]:
prompt_template = '''
Using the following snippet encased in ```, generate an image prompt of a robot that is inspired by a mix of Transformers, Gundam and Samurai armour, and is relevant to the snippet. The robot must take the shape of a random animal. Be detailed in the description of this robot. Ensure that the design and mechanism of it's body is very intricate, using a suitable 3 colour palette that includes black as a base with accents of the other 2 colours.

Also come up with a creative backstory in around 50 words on this robot's origin story, and include elements of the original snippet in the backstory.

```Sri Lanka Suffers Brief Power Outage After Main Transmission Line Fails```
'''  # noqa: E501

response_template = '''
Design Description:
Introducing "EnergiPanther," a robotic marvel blending Transformers, Gundam, and Samurai aesthetics. This sleek mechanized panther stands at 25 feet, adorned in intricate black armor with accents of deep crimson and electric gold. The samurai-inspired patterns on its chassis exude elegance, and its articulated limbs display a fusion of strength and precision.

In a majestic pose, EnergiPanther prowls, its golden eyes gleaming with determination. The crimson energy pulsating through its tail signifies the power coursing through its cybernetic veins.

Backstory:
Forged by visionary Sri Lankan engineers post a crucial power outage, EnergiPanther embodies the nation's spirit. The main transmission line failure spurred its creation, fusing traditional samurai ethos with cutting-edge robotics. EnergiPanther now stands as a guardian, symbolizing Sri Lanka's commitment to overcoming challenges and ensuring an uninterrupted power supply.'''

In [10]:
completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                'role': 'system',
                'content': 'You are a creative character designer'
            },
            {
                'role': 'user',
                'content': prompt_template
            },
            {
                'role': 'assistant',
                'content': response_template
            },
            {
                'role': 'user',
                'content': f'Do the same for the following snippet: `{selected_snippet}`'  # noqa: E501
            }
        ]
    )

image_prompt = completion.choices[0].message.content

print(selected_snippet)
print(image_prompt)

Palestinians report Israeli battles in Khan Younis after US blocks Gaza ceasefire call
Design Description:
Introducing "WarriorFalcon," a magnificent robotic creation influenced by Transformers, Gundam, and the ancient art of falconry. This avian-inspired robot stands tall, with a regal presence. Its sleek, black armor serves as the base, adorned with accents of vibrant green and metallic silver. Intricate patterns resembling feathers grace its body, showcasing meticulous craftsmanship.

WarriorFalcon stands poised, wings partially unfurled, ready to take flight. Its emerald eyes glow with fierce determination, and its silver talons gleam, showcasing its formidable strength and agility.

Backstory:
Engineered as a response to the escalating conflict between Israelis and Palestinians, WarriorFalcon embodies the resilience and determination of the Palestinian people. Inspired by the tales of ancient falconry, this robotic warrior combines traditional symbolism with cutting-edge technolog

In [11]:
prompt = image_prompt.replace('\n', '').split('Design Description:')[-1].split('Backstory:')[0]
post_text = image_prompt.replace('\n', '').split('Backstory:')[-1]

In [12]:
final_text = post_text

In [13]:
"""
elements_endpoint = "https://cloud.leonardo.ai/api/rest/v1/elements"

headers = {
    "accept": "application/json",
    "authorization": f"Bearer {leonardo_api_key}"
}

elements_response = requests.get(elements_endpoint, headers=headers)

elem_list = [(x['akUUID'], x['weightMax']) for x in elements_response.json()['loras']]
selected_elem = random.choice(elem_list)
"""

'\nelements_endpoint = "https://cloud.leonardo.ai/api/rest/v1/elements"\n\nheaders = {\n    "accept": "application/json",\n    "authorization": f"Bearer {leonardo_api_key}"\n}\n\nelements_response = requests.get(elements_endpoint, headers=headers)\n\nelem_list = [(x[\'akUUID\'], x[\'weightMax\']) for x in elements_response.json()[\'loras\']]\nselected_elem = random.choice(elem_list)\n'

#### Call Leonardo AI

In [14]:

url = "https://cloud.leonardo.ai/api/rest/v1/generations"

payload = {
    "modelId": "1e60896f-3c26-4296-8ecc-53e2afecc132",
    "prompt": prompt,
    "height": 512,
    "width": 512,
    "alchemy": True,
    "highResolution": True,
    "highResolution": True,
    "nsfw": True,
    "num_images": 1,
    "presetStyle": "CREATIVE",
    "public": True,
    "expandedDomain": True
}
headers = {
    "accept": "application/json",
    "content-type": "application/json",
    "authorization": f"Bearer {leonardo_api_key}"
}

response = requests.post(url, json=payload, headers=headers)

print(response.text)
for k in range(20):
    print(f'Sleeping...{k}')
    time.sleep(1)

{"sdGenerationJob":{"generationId":"f217b6b7-dc15-4837-bbb4-08a9231914d3","apiCreditCost":24}}
Sleeping...0
Sleeping...1


In [ ]:
response.json()

{'sdGenerationJob': {'generationId': '37c4ac26-2a79-4a91-80e2-43f0a36c796e',
  'apiCreditCost': 24}}

In [ ]:
gen_id = response.json()['sdGenerationJob']['generationId']

#### Grab the generated image from a CDN

In [ ]:
url = f"https://cloud.leonardo.ai/api/rest/v1/generations/{gen_id}"

headers = {
    "accept": "application/json",
    "authorization": f"Bearer {leonardo_api_key}"
}

response = requests.get(url, headers=headers)

print(response.text)

{"generations_by_pk":{"generated_images":[{"url":"https://cdn.leonardo.ai/users/8c647f87-f517-478a-95a7-e7d8f2dc1271/generations/37c4ac26-2a79-4a91-80e2-43f0a36c796e/Leonardo_Diffusion_XL_Introducing_Harmony_Guardian_a_striking_0.jpg","nsfw":false,"id":"40f79da4-9b8c-447d-8839-e5de8b5e61e3","likeCount":0,"generated_image_variation_generics":[]}],"modelId":"1e60896f-3c26-4296-8ecc-53e2afecc132","prompt":"Introducing \"Harmony Guardian,\" a striking robot inspired by a blend of Transformers, Gundam, and medieval knight aesthetics. This humanoid robot stands tall at 20 feet, exuding an air of strength and honor. Its intricate body design incorporates a primarily black armored frame highlighted with accents of cool blue and shimmering silver.Harmony Guardian's imposing appearance is reminiscent of a noble knight of old. Its sleek armor features intricate engravings that symbolize unity and diversity. The blue lines that trace its edges represent the importance of inclusivity, while the sil

In [ ]:
print(final_text)

Rejected by Wisconsin university regents, Harmony Guardian was initially meant to be a symbol of unity and progress. Created in collaboration between progressive-minded Republicans and the diverse university community, this robot was envisioned to serve as a reminder of the value of diversity positions. Despite the rejection, Harmony Guardian has become a beacon of hope in the hearts of those who believe in the power of inclusivity, aiming to inspire positive change within society.
